In [152]:
import import_ipynb
from tifffile import TiffFile
from tiff_to_png import page_split
from google.cloud import vision
from google.cloud.vision import types
import io
from PIL import Image, ImageDraw
from enum import Enum
import os
import cv2
import datetime
import json

In [153]:
path = input ("Give file name: ")

Give file name: doc.tiff


In [154]:
page_split (path)

tif = TiffFile(path)
num = len (tif.pages)
print (num)

8


In [155]:
def get_document_bounds(response, feature, document):
    bounds=[]
    for i,page in enumerate(document.pages):
        for block in page.blocks:
            #if feature==FeatureType.BLOCK:
            bounds.append(block.bounding_box)
            
            #for paragraph in block.paragraphs:
                #if feature==FeatureType.PARA:
             #   bounds.append(paragraph.bounding_box)
            '''for word in paragraph.words:
                    for symbol in word.symbols:
                        if (feature == FeatureType.SYMBOL):
                            bounds.append(symbol.bounding_box)
                    if (feature == FeatureType.WORD):
                        bounds.append(word.bounding_box)'''
    #print (bounds)
    return bounds

class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5

In [157]:

new_file, file_ex = os.path.splitext (path)

for i in range (num):
    
    image_file = path + "page%d.png" % i
    image  = Image.open(image_file)

    #new_files = datetime.datetime.now()
    
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'token.json'
    client = vision.ImageAnnotatorClient()
    with io.open(image_file, 'rb') as image_file1:
        content = image_file1.read()
    content_image = types.Image(content=content)
    response = client.document_text_detection(image=content_image)
    document1 = response.full_text_annotation
    document = response.full_text_annotation.text
    bound = get_document_bounds(response, FeatureType.PARA, document1)
    with open(str(new_file)+".txt", 'a') as f:
        print(response, file=f)
    with open(str(new_file)+".docx", 'a') as f:
        print(document+"\n\n\n\n\n", file=f)
    
    os.remove(image_file)
